In [1]:
%load_ext autoreload
%autoreload 2
import TRITON_SWMM_toolkit.examples as examples
import TRITON_SWMM_toolkit.analysis as analysis
from TRITON_SWMM_toolkit.process_simulation import TRITONSWMM_sim_post_processing
import TRITON_SWMM_toolkit.process_simulation as ps
import TRITON_SWMM_toolkit.processing_analysis as pe
from TRITON_SWMM_toolkit.process_simulation import *
import TRITON_SWMM_toolkit.plot_utils as plot_utils
import TRITON_SWMM_toolkit.utils as utils 
import TRITON_SWMM_toolkit.run_simulation as run
import TRITON_SWMM_toolkit.sensitivity_analysis as snst
from TRITON_SWMM_toolkit.config import analysis_config
from TRITON_SWMM_toolkit.examples import GetTS_TestCases as tst
from TRITON_SWMM_toolkit.scenario import TRITONSWMM_scenario
from importlib import reload
reload(examples)
reload(analysis)
reload(ps)
reload(pe)
reload(plot_utils)
reload(utils)
reload(snst)
rebuild_case = False

# single sim test case

In [2]:
testcase = tst.retreive_norfolk_single_sim_test_case(
        start_from_scratch=True, download_if_exists=False
    )


In [3]:
testcase.system.process_system_level_inputs()

In [4]:
testcase.system.compile_TRITON_SWMM(redownload_triton_swmm_if_exists=False, recompile_if_already_done_successfully=True)

In [11]:
self = testcase.system
compilation_log = self.retrieve_compilation_log()
swmm_check = "Built target swmm5" in compilation_log
triton_check = "[100%] Built target triton.exe" in compilation_log

In [13]:
print(self.retrieve_compilation_log())

-- Found Python3: /home/dcl3nd/miniconda3/envs/triton_swmm_toolkit/bin/python3 (found version "3.11.14") found components: Interpreter 
-- Detected FQDN: leedleedle.
-- machine file: /home/dcl3nd/dev/TRITON-SWMM_toolkit/test_data/norfolk_coastal_flooding/triton/cmake/machines/Linux/default_default.sh
-- TRITON_MACHINE=Linux
-- TRITON_COMPILER=mpic++
-- TRITON_BACKEND=OPENMP
-- TRITON_ARCH=NATIVE
-- TRITON_RUN_COMMAND=mpirun -n 8
-- The CXX compiler identification is GNU 13.3.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/mpic++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- CMAKE_CXX_COMPILER = /usr/bin/mpic++
-- SWMM coupling enabled - SWMM will be built from bundled source
-- Setting default Kokkos CXX standard to 17
-- Kokkos version: 4.5.99
-- The project name is: Kokkos
-- Using internal gtest for testing
-- Configured git information in /home/dcl3nd/dev/TRITON-SWMM

In [ ]:
scen = TRITONSWMM_scenario(0, analysis)
scen.log.scenario_creation_complete.get()
not scen.log.scenario_creation_complete.get()

In [ ]:
# def test_process_sim():
single_sim_single_core = tst.retreive_norfolk_single_sim_test_case()
analysis = single_sim_single_core.system.analysis
analysis.process_all_sim_timeseries_serially()
success_processing = (
    analysis.log.all_TRITON_timeseries_processed.get()
    and analysis.log.all_SWMM_timeseries_processed.get()
)
if not success_processing:
    analysis.print_logfile_for_scenario(0)
    print(f"Processing TRITON and SWMM time series failed.")
success_clearing = (
    analysis.log.all_raw_TRITON_outputs_cleared.get()
    and analysis.log.all_raw_SWMM_outputs_cleared.get()
)
if not success_clearing:
    analysis.print_logfile_for_scenario(0)
    print(f"Clearning raw outputs failed.")

# multi-sim testcase

In [ ]:
testcase = tst.retreive_norfolk_multi_sim_test_case(start_from_scratch=False)
analysis = testcase.system.analysis
self = analysis.process
assert analysis.log.TRITONSWMM_compiled_successfully.get()

In [ ]:
prepare_scenario_launchers = analysis.retrieve_prepare_scenario_launchers(
        overwrite_scenario=True, verbose=True
    )
analysis.run_python_functions_concurrently(prepare_scenario_launchers)

In [ ]:
analysis.log.all_scenarios_created.get()

In [ ]:
launch_functions = analysis._create_launchable_sims(
        pickup_where_leftoff=False, verbose=True
    )
analysis.run_simulations_concurrently_on_local_machine(launch_functions, verbose=True)

# sensitivity analysis

In [ ]:
nrflk_cpu_sensitivity = tst.retreive_norfolk_cpu_config_sensitivity_case(
        start_from_scratch=True
    )

In [ ]:
testcase = tst.retreive_norfolk_cpu_config_sensitivity_case(start_from_scratch=False)
analysis = testcase.system.analysis

In [ ]:
analysis.sensitivity.compile_TRITON_SWMM_for_sensitivity_analysis()
assert analysis.sensitivity.compilation_successful == True

In [ ]:
analysis.sensitivity.prepare_scenarios_in_each_subanalysis(concurrent=True)
assert analysis.log.all_scenarios_created.get() == True

In [ ]:
analysis.sensitivity.run_all_sims(pickup_where_leftoff=False, concurrent=False)
# assert analysis.log.all_sims_run.get() == True